<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc" style="margin-top: 1em;"><ul class="toc-item"><li><span><a href="#Search-for-a-reference-genome" data-toc-modified-id="Search-for-a-reference-genome-1">Search for a reference genome</a></span><ul class="toc-item"><li><span><a href="#Homo-sapiens's-reference-genome-sequence" data-toc-modified-id="Homo-sapiens's-reference-genome-sequence-1.1"><em>Homo sapiens</em>'s reference genome sequence</a></span></li><li><span><a href="#Mus-musculus's-reference-genome-sequence" data-toc-modified-id="Mus-musculus's-reference-genome-sequence-1.2"><em>Mus musculus</em>'s reference genome sequence</a></span></li><li><span><a href="#Saccharomyces-cerevisiae's-reference-genome-sequence" data-toc-modified-id="Saccharomyces-cerevisiae's-reference-genome-sequence-1.3"><em>Saccharomyces cerevisiae</em>'s reference genome sequence</a></span></li></ul></li><li><span><a href="#Download-from-the-NCBI" data-toc-modified-id="Download-from-the-NCBI-2">Download from the NCBI</a></span><ul class="toc-item"><li><span><a href="#List-of-chromosome/contigs" data-toc-modified-id="List-of-chromosome/contigs-2.1">List of chromosome/contigs</a></span></li><li><span><a href="#Sequences-of-each-chromosome/contigs" data-toc-modified-id="Sequences-of-each-chromosome/contigs-2.2">Sequences of each chromosome/contigs</a></span></li><li><span><a href="#Concatenate-all-contigs/chromosomes-into-single-files" data-toc-modified-id="Concatenate-all-contigs/chromosomes-into-single-files-2.3">Concatenate all contigs/chromosomes into single files</a></span></li></ul></li><li><span><a href="#Creation-of-an-index-file-for-GEM-mapper" data-toc-modified-id="Creation-of-an-index-file-for-GEM-mapper-3">Creation of an index file for GEM mapper</a></span><ul class="toc-item"><li><span><a href="#Extra:-Compute-mappability-for-oneD-normalization" data-toc-modified-id="Extra:-Compute-mappability-for-oneD-normalization-3.1">Extra: Compute mappability for oneD normalization</a></span></li></ul></li></ul></div>

In [1]:
import os

# Search for a reference genome

We would need two reference genomes. One as a FASTA file with each chromosome, and one that we will use exclusively for the mapping that would contain all contigs.

*The use of contigs in the reference genome increases the mapping specificity.*

## *Homo sapiens*'s reference genome sequence

Human (https://www.ncbi.nlm.nih.gov/genome/?term=Homo+sapiens%5Borgn%5D):

In [2]:
species  = 'Homo_sapiens'
taxid    = '9606'
assembly = 'GRCh38.p12'
genbank  = 'GCF_000001405.38'

## *Mus musculus*'s reference genome sequence

Mouse (https://www.ncbi.nlm.nih.gov/genome?term=mus%20musculus):

In [3]:
species  = 'Mus_musculus'
taxid    = '10090'
assembly = 'GRCm38.p6'
genbank  = 'GCF_000001635.26'

## *Saccharomyces cerevisiae*'s reference genome sequence

Yeast (https://www.ncbi.nlm.nih.gov/genome/?term=Saccharomyces+cerevisiae):

In [8]:
species = 'Saccharomyces_cerevisiae'
taxid   = '4932'
assembly = 'R64'
genbank = 'GCA_000146045.2'

# Download from the NCBI

##  List of chromosome/contigs

In [5]:
sumurl = ('ftp://ftp.ncbi.nlm.nih.gov/genomes/all/{0}/{1}/{2}/{3}/{4}_{5}/'
          '{4}_{5}_assembly_report.txt').format(genbank[:3], genbank[4:7], genbank[7:10], 
                                                genbank[10:13], genbank, assembly)

crmurl = ('https://eutils.ncbi.nlm.nih.gov/entrez/eutils/efetch.fcgi'
          '?db=nuccore&id=%s&rettype=fasta&retmode=text')

In [6]:
print sumurl

ftp://ftp.ncbi.nlm.nih.gov/genomes/all/GCA/000/146/045/GCA_000146045.2_R64/GCA_000146045.2_R64_assembly_report.txt


In [7]:
! wget -q $sumurl -O chromosome_list.txt

In [8]:
! head chromosome_list.txt

##  Sequences of each chromosome/contigs

In [14]:
dirname = 'genome'
! mkdir -p {dirname}

For each contig/chromosome download the corresponding FASTA file from NCBI

In [10]:
contig = []
for line in open('chromosome_list.txt'):
    if line.startswith('#'):
        continue
    seq_name, seq_role, assigned_molecule, _, genbank, _, refseq, _ = line.split(None, 7)
    if seq_role == 'assembled-molecule':
        name = 'chr%s.fasta' % assigned_molecule
    else:
        name = 'chr%s_%s.fasta' % (assigned_molecule, seq_name.replace('/', '-'))
    contig.append(name)

    outfile = os.path.join(dirname, name)
    if os.path.exists(outfile) and os.path.getsize(outfile) > 10:
        continue
    error_code = os.system('wget "%s" --no-check-certificate -O %s' % (crmurl % (genbank), outfile))
    if error_code:
        error_code = os.system('wget "%s" --no-check-certificate -O %s' % (crmurl % (refseq), outfile))
    if error_code:
        print genbank

## Concatenate all contigs/chromosomes into single files

In [11]:
def write_to_fasta(line):
    contig_file.write(line)

def write_to_fastas(line):
    contig_file.write(line)
    simple_file.write(line)

In [12]:
os.system('mkdir -p {}/{}-{}'.format(dirname, species, assembly))

0

In [13]:
contig_file = open('{0}/{1}-{2}/{1}-{2}_contigs.fa'.format(dirname, species, assembly),'w')
simple_file = open('{0}/{1}-{2}/{1}-{2}.fa'.format(dirname, species, assembly),'w')

for molecule in contig:
    fh = open('{0}/{1}'.format(dirname, molecule))
    oline = '>%s\n' % (molecule.replace('.fasta', ''))
    _ = fh.next()
    # if molecule is an assembled chromosome we write to both files, otherwise only to the *_contigs one
    write = write_to_fasta if '_' in molecule else write_to_fastas
    for line in fh:
        write(oline)
        oline = line
    # last line usually empty...
    if line.strip():
        write(line)
contig_file.close()
simple_file.close()

Remove all the other files (with single chromosome/contig)

In [14]:
! rm -f {dirname}/*.fasta

# Creation of an index file for GEM mapper

In [15]:
! gem-indexer -t 8 -i {dirname}/{species}-{assembly}/{species}-{assembly}_contigs.fa -o {dirname}/{species}-{assembly}/{species}_contigs

Welcome to GEM-indexer build 1.335 (beta) - (2012-10-29 22:24:35 GMT)
 (c) 2008-2012 Paolo Ribeca <paolo.ribeca@gmail.com>
 (c) 2010-2012 Santiago Marco Sola <santiagomsola@gmail.com>
 (c) 2010-2012 Leonor Frias Moya <leonor.frias@gmail.com>
For the terms of use, run the program with the option --show-license.
************************************************************************
* WARNING: this is a beta version, provided for testing purposes only; *
*          check for updates at <http://gemlibrary.sourceforge.net>.   *
************************************************************************
Creating sequence and location files... done.
Computing DNA BWT (likely to take long)... done.
Generating index (likely to take long)... done.
Cleaning up... done.


The index file will be: __`{dirname}/{species}-{assembly}/{species}_contigs.gem`__

## Extra: Compute mappability for oneD normalization

In this case we can use the FASTA of the genome whithout contigs and follow these step:

In [ ]:
! gem-indexer -i {dirname}/{species}-{assembly}/{species}-{assembly}.fa \
   -o {dirname}/{species}-{assembly}/{species}-{assembly} -t 8

! gem-mappability -I {dirname}/{species}-{assembly}/{species}-{assembly}.gem -l 50 \
   -o {dirname}/{species}-{assembly}/{species}-{assembly}.50mer -T 8

! gem-2-wig -I {dirname}/{species}-{assembly}/{species}-{assembly}.gem \
   -i {dirname}/{species}-{assembly}/{species}-{assembly}.50mer.mappability \
   -o {dirname}/{species}-{assembly}/{species}-{assembly}.50mer

! wigToBigWig {dirname}/{species}-{assembly}/{species}-{assembly}.50mer.wig \
   {dirname}/{species}-{assembly}/{species}-{assembly}.50mer.sizes Saccharomyces_cerevisiae.50mer.bw

! bigWigToBedGraph {dirname}/{species}-{assembly}/{species}-{assembly}.50mer.bw  \
   {dirname}/{species}-{assembly}/{species}-{assembly}.50mer.bedGraph

Welcome to GEM-indexer build 1.335 (beta) - (2012-10-29 22:24:35 GMT)
 (c) 2008-2012 Paolo Ribeca <paolo.ribeca@gmail.com>
 (c) 2010-2012 Santiago Marco Sola <santiagomsola@gmail.com>
 (c) 2010-2012 Leonor Frias Moya <leonor.frias@gmail.com>
For the terms of use, run the program with the option --show-license.
************************************************************************
* WARNING: this is a beta version, provided for testing purposes only; *
*          check for updates at <http://gemlibrary.sourceforge.net>.   *
************************************************************************
Creating sequence and location files... done.
Computing DNA BWT (likely to take long)... done.
Generating index (likely to take long)... done.
Cleaning up... done.
Welcome to GEM-mappability build 1.315 (beta) - (2013/03/29 02:59:40 GMT)
 (c) 2008-2013 Paolo Ribeca <paolo.ribeca@gmail.com>
 (c) 2010-2013 Santiago Marco Sola <santiagomsola@gmail.com>
 (c) 2010-2013 Leonor Frias Moya <leonor.fri

limit_output extension: Maximum message size of 20000 exceeded with 20038 characters